In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt


The task is to connect to an API that plays a Wordle-like puzzle. You can find the API documentation here: [Wordle API Documentation] https://wordle.votee.dev:8000/redoc

Your task is to write a program that automatically guesses random words. You may use any frameworks, IDEs, chatbots, LLMs, or other resources to write your program and showcase your skills. 

wordseg

In [17]:
import random
import requests

# API base URL
BASE_URL = "https://wordle.votee.dev:8000"
def word_seg(text):
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {"text": text}
    response = requests.post(f"{BASE_URL}/wordseg", headers=headers,data=data)
    
    if response.status_code == 200:   
        feedback = response.json()
        print(f"Wordseg Response for text: '{text}'")
        print(feedback)
        
        return feedback
    else:
        print("Error guessing the daily puzzle with guess:",response.text)
        return None
    # except requests.exceptions.RequestException as e:
    #     print(f"Error guessing the word '{word}' with guess '{guess}': {e}")
    #     return None


if __name__ == "__main__":
    print("word seg...")
    sample_text="Testing word segmentation"
    # daily_guess="delta"
    word_seg(sample_text)

word seg...
Wordseg Response for text: 'Testing word segmentation'
['▁testing', '▁word', '▁segment', 'ation']


Guess daily

In [27]:
import random
import requests
import string
# API base URL
BASE_URL = "https://wordle.votee.dev:8000"

def generate_random_word(size,known_chars,incorrect_chars):
    """
    Generate a random word of the specified size.
    Args:
        size (int): Length of the word.
    Returns:
        str: A randomly generated word.
    """
    # return ''.join(random.choices(string.ascii_lowercase, k=size))
    word = [''] * size  # Initialize word with empty characters

    # Fill in the known characters first
    for position, char in known_chars.items():
        word[position] = char

    # Generate remaining characters randomly
    remaining_positions = [i for i in range(size) if word[i] == '']
    for pos in remaining_positions:
        # Exclude both correct guesses and incorrect guesses from the random selection pool
        excluded_chars = set(known_chars.values()).union(incorrect_chars.get(pos, set()))
        remaining_chars = [c for c in string.ascii_lowercase if c not in excluded_chars]
        word[pos] = random.choice(remaining_chars)
        # # Exclude correct guesses from the random selection pool
        # remaining_chars = [c for c in string.ascii_lowercase if c not in known_chars.values()]
        # word[pos] = random.choice(remaining_chars)

    return ''.join(word)

def guess_daily_puzzle(size):
    attempts=0
    solved=False
    max_attempt=1000
    known_chars={}
    incorrect_chars={}
    while not solved and attempts<max_attempt:
        guess= generate_random_word(size,known_chars,incorrect_chars)
        params = {"guess": guess,"size":size}
        response = requests.get(f"{BASE_URL}/daily", params=params)

        if response.status_code == 200:   
            feedback = response.json()
            attempts+=1
            print(f"Daily Puzzle | Attempt {attempts}  Guess: '{guess}'")
            # print(feedback)
            correct_count=0
            for item in feedback:
                print(f"Slot: {item['slot']}, Letter: '{item['guess']}', Result: {item['result']}")
                if item['result']=="correct":
                    correct_count+=1
                    known_chars[item['slot']] = item['guess'] 
                elif item['result'] == "incorrect":
                    # Add incorrect characters to the dictionary for future exclusion
                    if item['slot'] not in incorrect_chars:
                        incorrect_chars[item['slot']] = set()
                    incorrect_chars[item['slot']].add(item['guess'])
            # Check if the puzzle is solved
            if correct_count == size:
                print(f"🎉 Puzzle solved in {attempts} attempts! The word is '{guess}'.")
                solved = True
            # return feedback
        else:
            print("Error guessing the daily puzzle with guess:",response.text)
            break
            # return None
    # except requests.exceptions.RequestException as e:
    #     print(f"Error guessing the word '{word}' with guess '{guess}': {e}")
    #     return None


if __name__ == "__main__":
    print("Guessing the daily puzzle...")
    # daily_guess="delta"
    guess_daily_puzzle(5)

Guessing the daily puzzle...
Daily Puzzle | Attempt 1  Guess: 'lcdrt'
Slot: 0, Letter: 'l', Result: absent
Slot: 1, Letter: 'c', Result: absent
Slot: 2, Letter: 'd', Result: absent
Slot: 3, Letter: 'r', Result: present
Slot: 4, Letter: 't', Result: absent
Daily Puzzle | Attempt 2  Guess: 'maigm'
Slot: 0, Letter: 'm', Result: absent
Slot: 1, Letter: 'a', Result: correct
Slot: 2, Letter: 'i', Result: present
Slot: 3, Letter: 'g', Result: absent
Slot: 4, Letter: 'm', Result: absent
Daily Puzzle | Attempt 3  Guess: 'vaotm'
Slot: 0, Letter: 'v', Result: absent
Slot: 1, Letter: 'a', Result: correct
Slot: 2, Letter: 'o', Result: absent
Slot: 3, Letter: 't', Result: absent
Slot: 4, Letter: 'm', Result: absent
Daily Puzzle | Attempt 4  Guess: 'jahyn'
Slot: 0, Letter: 'j', Result: absent
Slot: 1, Letter: 'a', Result: correct
Slot: 2, Letter: 'h', Result: absent
Slot: 3, Letter: 'y', Result: absent
Slot: 4, Letter: 'n', Result: absent
Daily Puzzle | Attempt 5  Guess: 'dacrd'
Slot: 0, Letter: 'd',

guess random word

In [18]:
import random
import requests

# API base URL
BASE_URL = "https://wordle.votee.dev:8000"

# Wordlist for random guessing
WORD_LIST = ["apple", "berry", "chess", "delta", "fuzzy", "grape", "heart", "jumpy", "knock", "lunar"]
# use generate word list for random guessing
def generate_random_word_list(size, count=50):
    word_list = [''.join(random.choices(string.ascii_lowercase, k=size)) for _ in range(count)]
    return word_list

def guess_random_word(size=5, seed=None):
    params = {"size": size}
    if seed is not None:
        params["seed"] = seed

    # Choose a random word from the word list as the guess
    word_length=size
    word_list=generate_random_word_list(word_length, count=40)
    guess_word=random.choice(word_list)

    # guess_word = random.choice([word for word in WORD_LIST if len(word) == size])
    params["guess"] = guess_word

    response = requests.get(f"{BASE_URL}/random", params=params)
    if response.status_code == 200:
        feedback = response.json()
        print(f"Guess: '{guess_word}'")
        for item in feedback:
            print(f"Slot: {item['slot']}, Letter: '{item['guess']}', Result: {item['result']}")
        return feedback
    else:
        print("Failed to make a random guess:", response.text)
        return None

if __name__ == "__main__":
    print("Making a random word guess...")
    guess_random_word(size=5, seed=42)
# def guess_random_word(word_list):
#     secret_word=random.choice(word_list)
#     guessed_word=None  
#     while guess_random_word !=secret_word:
#         guessed_word=random.choice(word_list)
#         print()

Making a random word guess...
Guess: 'swnlg'
Slot: 0, Letter: 's', Result: absent
Slot: 1, Letter: 'w', Result: present
Slot: 2, Letter: 'n', Result: absent
Slot: 3, Letter: 'l', Result: absent
Slot: 4, Letter: 'g', Result: absent


guess selected word

In [29]:
import random
import requests
import string

# API base URL
BASE_URL = "https://wordle.votee.dev:8000"

# Wordlist for random guessing
WORD_LIST = ["apple", "berry", "chess", "delta", "fuzzy", "grape", "heart", "jumpy", "knock", "lunar"]

# Generate random word list
def generate_random_word_list(size, count=50, known_chars={}, incorrect_chars={}):
    word_list = []
    
    for _ in range(count):
        word = [''] * size  # Initialize word with empty characters

        # Fill in the known characters first
        for position, char in known_chars.items():
            word[position] = char

        # Generate remaining characters randomly
        remaining_positions = [i for i in range(size) if word[i] == '']
        for pos in remaining_positions:
            # Exclude both correct guesses and incorrect guesses from the random selection pool
            excluded_chars = set(known_chars.values()).union(incorrect_chars.get(pos, set()))
            remaining_chars = [c for c in string.ascii_lowercase if c not in excluded_chars]
            word[pos] = random.choice(remaining_chars)

        word_list.append(''.join(word))

    return word_list

def guess_selected_word(word):
    word_length = len(word)
    known_chars = {}  # Dictionary to track correct characters and their positions
    incorrect_chars = {}  # Dictionary to track incorrect characters and their positions

    attempts = 0
    solved = False
    while not solved and attempts < 1000:  # Limiting to 10 attempts
        word_list = generate_random_word_list(word_length, count=40, known_chars=known_chars, incorrect_chars=incorrect_chars)
        random_guess = random.choice(word_list)

        params = {"guess": random_guess}
        response = requests.get(f"{BASE_URL}/word/{word}", params=params)

        if response.status_code == 200:
            feedback = response.json()
            attempts += 1
            print(f"Target Word: '{word}' | Attempt {attempts} | Guess: '{random_guess}'")

            correct_count = 0
            for item in feedback:
                print(f"Slot: {item['slot']}, Letter: '{item['guess']}', Result: {item['result']}")
                if item['result'] == "correct":
                    correct_count += 1
                    known_chars[item['slot']] = item['guess']  # Update known correct characters
                elif item['result'] == "incorrect":
                    # Add incorrect characters to the dictionary for future exclusion
                    if item['slot'] not in incorrect_chars:
                        incorrect_chars[item['slot']] = set()
                    incorrect_chars[item['slot']].add(item['guess'])

            # Check if the puzzle is solved
            if correct_count == word_length:
                print(f"🎉 Puzzle solved in {attempts} attempts! The word is '{random_guess}'.")
                solved = True
        else:
            print("Failed to make a random guess:", response.text)
            break

if __name__ == "__main__":
    print("Guessing the selected word...")
    target = "peach"
    guess_selected_word(target)


Guessing the selected word...
Target Word: 'peach' | Attempt 1 | Guess: 'vocyx'
Slot: 0, Letter: 'v', Result: absent
Slot: 1, Letter: 'o', Result: absent
Slot: 2, Letter: 'c', Result: present
Slot: 3, Letter: 'y', Result: absent
Slot: 4, Letter: 'x', Result: absent
Target Word: 'peach' | Attempt 2 | Guess: 'vducw'
Slot: 0, Letter: 'v', Result: absent
Slot: 1, Letter: 'd', Result: absent
Slot: 2, Letter: 'u', Result: absent
Slot: 3, Letter: 'c', Result: correct
Slot: 4, Letter: 'w', Result: absent
Target Word: 'peach' | Attempt 3 | Guess: 'jivce'
Slot: 0, Letter: 'j', Result: absent
Slot: 1, Letter: 'i', Result: absent
Slot: 2, Letter: 'v', Result: absent
Slot: 3, Letter: 'c', Result: correct
Slot: 4, Letter: 'e', Result: present
Target Word: 'peach' | Attempt 4 | Guess: 'dwvct'
Slot: 0, Letter: 'd', Result: absent
Slot: 1, Letter: 'w', Result: absent
Slot: 2, Letter: 'v', Result: absent
Slot: 3, Letter: 'c', Result: correct
Slot: 4, Letter: 't', Result: absent
Target Word: 'peach' | At